# Risk Simulation for SC,AU,CF,SR and OI 2018-2023

Simulation method: 
- Calc weighted close price for all futures
- Find top 1%, 5%, 10% d2d diff and corresponding dates



In [16]:
import os
from process_data import run

# run()

def get_xls_files():
    xchanges = ['ine', 'shfe']
    file_paths = []
    for xchange in xchanges:
        for root, directories, files in os.walk(f'./csv/{xchange}'):
            for filename in files:
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)
    return file_paths

def get_txt_files():
    xchanges = ['czce']
    file_paths = []
    for xchange in xchanges:
        for root, directories, files  in os.walk(f'./csv/{xchange}'):
            for filename in files:
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)
    return file_paths

shfe = get_xls_files()
czce = get_txt_files()

In [20]:
import pandas as pd
import numpy as np

und_shfe = ['sc', 'au']
und_czce = ['SR', 'CF', 'OI']

def get_shfe(filePath):
    df = pd.read_csv(filePath)
    d = df[df['Contract'].str.len() == 6]
    d['Und'] = df.Contract.str[:2]
    dd = d.groupby(['Date', 'Und']).apply(
        lambda x: pd.Series([
            np.ma.average(x['Close'], weights=x['OI'])
        ],index=['WeightedAvgClose'])
    ).reset_index()
    dd['Date'] = pd.to_datetime(dd['Date'], format='%Y%m%d')
    return dd

def get_czce(filePath):
    df = pd.read_csv(filePath, encoding='gb2312')
    df.rename(columns={'交易日期':'Date', '品种代码': 'Contract', '今收盘': 'Close', '成交量(手)': 'OI'}, inplace=True)
    df['Und'] = df.Contract.str[:2]
    dd = df.groupby(['Date', 'Und']).apply(
        lambda x: pd.Series([
            np.ma.average(x['Close'], weights=x['OI'])
        ],index=['WeightedAvgClose'])
    ).reset_index()
    dd['Date'] = pd.to_datetime(dd['Date'], format='%Y-%m-%d')
    return dd

def convert_to_lod(df, unds):
    lod = []
    for und in unds:
        d = df[df['Und'] == und]
        if d.__len__() != 0:
            lod.extend(d.to_dict('records'))
    return lod



In [21]:
df = get_czce(czce[0])

ZeroDivisionError: Weights sum to zero, can't be normalized

In [9]:
d = df[df['Contract'].str.len() == 6]
d['Und'] = df.Contract.str[:2]
dd = d.groupby(['Date', 'Und']).apply(
    lambda x: pd.Series([
        np.average(x['Close'], weights=x['OI'])
    ],index=['WeightedAvgClose'])
).reset_index()
dd = dd[dd['Und'] == 'au']

C:\Users\alan\AppData\Local\Temp\ipykernel_15108\1477275319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['Und'] = df.Contract.str[:2]


0